# Broker

In [1]:
#options for single values
value_options= [
                ('days_v',int()),
]

#dictionary where we find single values which we need to publish 
values = dict()
for opt in value_options:
  values[opt[0]] = opt[1]

#OPTIONS FOR SLIDER AND DROPDOWNS
slider_options = [
          ('età',18,100,'i'),
          ('PCR',0,500,'f'),
          ('LDH',100,1000,'f'),
          ('WBC',0,400,'f'),
          ('D-Dimero',100,30000,'f'),
          ('linfociti (%)',1,12,'f'),
          ('neutrofili/linfociti',0,150,'f'),
          #('f8',1,12)
          ]

dropdown_options = [
          ('risultato tampone',['positivo','debolmente positivo','negativo','non valido']),                   
          ['sesso',['Maschio','Femmina']] 
         ]

#values for sliders and dropdowns 
# slider_values[name_of_slider] = value
#dropdown_values[name_of_dropdown] = value
slider_values = dict()
dropdown_values = dict()
for opt in slider_options:
 # print(opt)
  slider_values[str(opt[0])] = float()
for opt in dropdown_options:
  dropdown_values[str(opt[0])] = str()

# We define a list of object wich is called when there's a change in a group of variables
sub_dashboard=list()
sub_days=list()

cells=dict()

# Import and utils

In [2]:
import ipywidgets as widgets
from IPython.display import display,clear_output

def space():
  return widgets.HTML('<br>')

def line():
  return widgets.HTML('<hr>')

def see_values():
  print(slider_values)
  print(dropdown_values)
  print(values)
  print(sub_dashboard)
  print(sub_days)

def change_value(name_variable,value,dict_to_change,subscribers):
  dict_to_change[name_variable] = value
  #we define mathod update wich update all the variables in the subscriber
  #post method on the contrary change value in the broker
  for sub in subscribers:
    sub.update()   

def subscribe(obj,sub):
  sub.append(obj)  

# all cells must have compose method to visualize
def see(obj,compose=True):
  if compose:
   display(widgets.VBox([space(),obj.compose()]))   
  else:
   display(widgets.VBox([space(),obj]))     

# Cells

In [3]:
class Days_toggle():
 def post_day(self,change):
   change_value('days_v',self.days.value,self.val,self.sub_days)
   
  
 def __init__(self,values,sub_days):
  self.val = values
  self.sub_days = sub_days
  self.days = widgets.ToggleButtons(options=['2', '4', '6','8','10','end'],layout=widgets.Layout(width='95%'))
  self.l_day = widgets.Label(value='giorni ospedalizzazione',layout=widgets.Layout(display="flex", justify_content="center"))
  self.days.observe(self.post_day, names='value')
  

 def compose(self):
  return widgets.VBox([self.l_day,self.days])

########################################################################################################################

d = Days_toggle(values,sub_days)
#display(widgets.VBox([space(),d.compose()]))
see(d)


In [11]:

class Dashboard:
 def post_dashboard_values(self,change=None):
  for w in self.sliders:
    change_value(w['l_f'].value,w['widget'].value,self.slider_val,self.sub_dashboard)
  for w in self.dropdowns:
    change_value(w['l_f'].value,w['widget'].value,self.dropdown_val,self.sub_dashboard)

 def null_variation(self,change):
  for w in self.sliders:
    w['widget'].value = w['widget'].min
  self.post_dashboard_values() 

 def __init__(self,slider_options,dropdown_options,slider_values,dropdown_values,sub_dashboard):
  
  self.sub_dashboard=sub_dashboard
  self.sliders = list()
  self.dropdowns = list() 
  self.slider_val = slider_values
  self.dropdown_val = dropdown_values

  self.b_reset = widgets.Button(description='Azzera Valori')

  for op in slider_options:
   f = dict()
   f['widget'] = widgets.FloatSlider(min=op[1],max=op[2])
   f['l_f'] = widgets.Label(value=op[0])
   #f['p_f'] = widgets.Label()
   self.sliders.append(f)

  for op in dropdown_options:
   f = dict()
   f['widget'] = widgets.Dropdown(options=op[1])
   f['l_f']= widgets.Label(value=op[0])
   #f['p_f'] = widgets.Label()
   self.dropdowns.append(f)

  for w in self.sliders + self.dropdowns:
   w['widget'].observe(self.post_dashboard_values, names='value')

  self.b_reset.on_click(self.null_variation)        

 def compose(self): 
  cells = list()
  for s in self.sliders + self.dropdowns:
   cells.append(widgets.VBox([s['l_f'],s['widget']]))
        
   columns = [widgets.VBox(cells[:3]),
             widgets.VBox(cells[3:6]),
             widgets.VBox(cells[6:]),
             widgets.VBox([self.b_reset],layout=widgets.Layout(display="flex", justify_content="flex-end"))]


  return widgets.VBox([space(),
                       widgets.HBox(columns,layout=widgets.Layout(display="flex", justify_content="center")),
                       space()],layout=widgets.Layout(border='1px solid grey'))

######################################################################################################## 

dash = Dashboard(slider_options,dropdown_options,slider_values,dropdown_values,sub_dashboard)
cell_dashboard = dash.compose()
see(dash)


In [5]:
class SendButton():
  
 def post(self):
  return

 def __init__(self):
   self.b_send = widgets.Button(description='Invia',layout=widgets.Layout(width='100%',height='30%')) 
   self.b_send.on_click(self.post()) 
   
 def compose(self):
   return self.b_send

############################################################################

b = SendButton()
see(b)

In [6]:
class Preview():

  #method called update are called if a cell is a sub of another value
  def update(self):
   for k in list(self.slider_values.keys()) + list(self.dropdown_values.keys()):
     for l in self.labels:
       if k == l['l_f']:
         if k in self.slider_values.keys():
          l['widget'].value = str(k) + ': ' + str(self.slider_values[k])
         else:
          l['widget'].value = str(k) + ': ' + str(self.dropdown_values[k])  


  def __init__(self,slider_values,dropdown_values):

   self.slider_values = slider_values
   self.dropdown_values=dropdown_values
   self.labels=list()
   self.update()
   
   for k in self.slider_values.keys():
    f = dict()
    f['l_f'] = k
    f['widget'] = widgets.Label(value=str(k) + ': ' + str(self.slider_values[k]))
    self.labels.append(f)

   for k in self.dropdown_values.keys():
    f = dict()
    f['l_f'] = k
    f['widget'] = widgets.Label(value=str(k) + ': ' + str(self.dropdown_values[k]))
    self.labels.append(f) 
   
  def compose(self):
   rows = [widgets.Label(value='Dati Paziente:')]
   
   for l in self.labels:
     rows.append(l['widget'])
     
   return widgets.Box([space(),
                       widgets.VBox(rows,layout=widgets.Layout(display="flex", justify_content="center")),
                       space()],
                      layout=widgets.Layout(border='1px solid grey'))
       
  
#############################################################################################        
p = Preview(slider_values,dropdown_values)
subscribe(p,sub_dashboard)
see(p)   


In [7]:
class Output():
  def __init__(self):
    self.res ={True: widgets.Label(value='Alta probabilità di guarigione'), 
               False: widgets.Label(value='Bassa probabilità di guarigione')}
    
    ##TODO grafo a torta quando ci saranno i dati

In [8]:
#!tar -xvzf mdl.tar.gz

# Nuova sezione

In [9]:
import dill
from sklearn.pipeline import Pipeline
def load_classification_tourney():
 #!tar -xvzf mdl.tar.gz   
 filepath = 'classification_tourney_mortality_risk_covid19_all_patients_simplified.dill'
 dill_file = open(filepath, 'rb')
 classification_tourney = dill.load(dill_file)
 dill_file.close()
 return classification_tourney



#model to use for each dataset. They are been chosen especially for recall and f2 score 
day_model={'2':'XGBC','4':'XGBC','6':'LGBM','8':'LGBM','10':'XGBC','end':'XGBC'}
day_data={'2':'2days','4':'4days','6':'6days','8':'8days','end':'end'}

def get_model_id(day,day_model):
    return day_model[str(day)] + '_iterativeimputer_randomsearch_10'

def get_dataset_id(day,day_data):
    return day_data[str(day)] + '_all_patients_numerical_simplified_3plus7_features'


def get_solution(tourney,day,day_data,day_model):
    return tourney.classification_matches[(get_dataset_id(2,day_data)),get_model_id(2,day_model)].results.solution
 
        
tourney = load_classification_tourney()


In [10]:
 solution: Pipeline = self.classifier_solution.create_pipeline_randomcv(
            ds=self.dataset.train.x
        )
        # fit the custom solution
        solution.fit(X=self.dataset.train.x, y=self.dataset.train.y)

        # Get the prediction for the test set
        y_test_predicted = solution.predict(self.dataset.test.x)


IndentationError: unexpected indent (1192507962.py, line 5)